In [ ]:
!pip install accelerate peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile

zip_path = '/content/drive/MyDrive/ukr_lit.zip'
extract_path = '/content/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


Mounted at /content/drive


In [ ]:
import os

content_path = "/content/data/ukr_lit/texts/1"
all_text = ""

# Сортуємо, якщо потрібен порядок
for filename in sorted(os.listdir(content_path)):
    if filename.endswith('.txt'):
        with open(os.path.join(content_path, filename), 'r', encoding='utf-8') as f:
            content = f.read()
            # print(f"content {content}")
            all_text += content + "\n\n<|sep|>\n\n"

# with open("dataset.txt", "w", encoding="utf-8") as out_file:
#     out_file.write(all_text)

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
from datasets import Dataset
import re

samples = all_text.split("<|sep|>")

# Очищаємо зайві пробіли
samples = [
    re.sub(r'\n+', ' ', re.sub(r'\n+Тарас Шевченко\n+', ' ', s))  # Спочатку прибираємо "Тарас Шевченко", потім заміняємо всі переноси на пробіл
    for s in samples
]

print(len(samples))

dataset = Dataset.from_dict({
    "text": samples
})

270


In [ ]:
from sklearn.model_selection import train_test_split

shuffled_dataset = dataset.shuffle(seed=42)
train_dataset, val_dataset = shuffled_dataset.train_test_split(test_size=0.1).values()

In [ ]:
dataset[0]

{'text': ' Злоначинающих спини... Злоначинающих спини...  Злоначинающих спини, У пута кутії не куй, В склепи глибокі не муруй. А доброзиждущим рукам І покажи, і поможи, Святую силу ниспошли. А чистих серцем? Коло їх Постави ангели свої І чистоту їх соблюди. А всім нам вкупі на землі Єдиномисліє подай І братолюбіє пошли. (27 травня 1860 р., С.— Петербург)   '}

In [ ]:
# from transformers import GPT2Tokenizer, GPT2LMHeadModel

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("malteos/gpt2-uk")
model = AutoModelForCausalLM.from_pretrained("malteos/gpt2-uk")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512, add_special_tokens=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/243 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as TorchDataset
import torch

class CustomDataset(TorchDataset):
    def __init__(self, tokenizer, dataset):
        self.tokenizer = tokenizer
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        input_ids = torch.tensor(item['input_ids'])
        attention_mask = torch.tensor(item['attention_mask'])
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

# Створюємо датасет PyTorch
train_dataset = CustomDataset(tokenizer, tokenized_train_dataset)
val_dataset = CustomDataset(tokenizer, tokenized_val_dataset)

batch_size=8

# Створюємо DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Новий розділ

In [ ]:
import torch
import math

def calculate_perplexity(model, val_dataloader, device):
    model.to(device)
    model.eval()  # Перемикаємо модель на режим оцінки
    total_loss = 0
    with torch.no_grad():  # Вимикаємо обчислення градієнтів
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Прямий прохід
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item()

    # Обчислюємо Perplexity
    avg_loss = total_loss / len(val_dataloader)
    perplexity = math.exp(avg_loss)  # Експоненціємо середні втрати
    return perplexity

# Приклад використання
perplexity = calculate_perplexity(model, val_dataloader,  'cuda')
print(f"Perplexity: {perplexity}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Perplexity: 13810.283747611735


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_score(model, dataloader, device):
    model.eval()  # Перемикаємо модель на режим оцінки
    references = []
    hypotheses = []

    with torch.no_grad():  # Вимикаємо обчислення градієнтів
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Генерація тексту
            outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Додаємо еталонні та згенеровані тексти
            reference_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            references.append([reference_text])  # Зберігаємо як список рядків
            hypotheses.append(generated_text)  # Зберігаємо як рядок

    # Обчислюємо BLEU score для всього набору даних
    smoothing_function = SmoothingFunction().method4  # Можна змінити метод згладжування
    bleu_score = sentence_bleu(references, hypotheses, smoothing_function=smoothing_function)
    return bleu_score

# Приклад використання
bleu_score = calculate_bleu_score(model, val_dataloader, 'cuda')
print(f"BLEU Score: {bleu_score}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BLEU Score: 0


# Новий розділ

In [ ]:
import torch.optim as optim
from tqdm import tqdm

optimizer = optim.Adam(model.parameters(), lr=5e-5)
device = "cuda"
num_epochs = 3

def evaluate_model(model, val_dataloader, device):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass on the validation set
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            val_loss += loss.item()

    # Print the average validation loss
    return val_loss / len(val_dataloader)

def train_model(model, train_dataloader, optimizer, num_epochs, device):
    model.to(device)  # Move model to the appropriate device (GPU or CPU)
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        epoch_loss = 0  # To track loss for the entire epoch
        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            # Backward pass
            optimizer.zero_grad()  # Zero gradients before backward pass
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update the model's parameters

            epoch_loss += loss.item()  # Accumulate loss for the epoch

        # Print the average loss for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs} | Average Loss: {epoch_loss / len(train_dataloader)}")

        val_loss = evaluate_model(model, val_dataloader, device)
        print(f"Epoch {epoch + 1}/{num_epochs} | Validation Loss: {val_loss}")


train_model(model, train_dataloader, optimizer, num_epochs, device)

Training Epoch 1/3: 100%|██████████| 31/31 [00:31<00:00,  1.00s/it]


Epoch 1/3 | Average Loss: 3.7422809024010935
Epoch 1/3 | Validation Loss: 3.092914581298828


Training Epoch 2/3: 100%|██████████| 31/31 [00:30<00:00,  1.00it/s]


Epoch 2/3 | Average Loss: 2.2431732223879908
Epoch 2/3 | Validation Loss: 3.008704125881195


Training Epoch 3/3: 100%|██████████| 31/31 [00:30<00:00,  1.02it/s]


Epoch 3/3 | Average Loss: 1.9820675003913142
Epoch 3/3 | Validation Loss: 3.0088483691215515


In [ ]:
import torch
import math

def calculate_perplexity(model, val_dataloader, device):
    model.eval()  # Перемикаємо модель на режим оцінки
    total_loss = 0
    with torch.no_grad():  # Вимикаємо обчислення градієнтів
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Прямий прохід
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item()

    # Обчислюємо Perplexity
    avg_loss = total_loss / len(val_dataloader)
    perplexity = math.exp(avg_loss)  # Експоненціємо середні втрати
    return perplexity

# Приклад використання
perplexity = calculate_perplexity(model, val_dataloader, device)
print(f"Perplexity: {perplexity}")

Perplexity: 20.26404977700014


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_score(model, dataloader, device):
    model.eval()  # Перемикаємо модель на режим оцінки
    references = []
    hypotheses = []

    with torch.no_grad():  # Вимикаємо обчислення градієнтів
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Генерація тексту
            outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Додаємо еталонні та згенеровані тексти
            reference_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            references.append([reference_text])  # Зберігаємо як список рядків
            hypotheses.append(generated_text)  # Зберігаємо як рядок

    # Обчислюємо BLEU score для всього набору даних
    smoothing_function = SmoothingFunction().method4  # Можна змінити метод згладжування
    bleu_score = sentence_bleu(references, hypotheses, smoothing_function=smoothing_function)
    return bleu_score

# Приклад використання
bleu_score = calculate_bleu_score(model, val_dataloader, device)
print(f"BLEU Score: {bleu_score}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BLEU Score: 0.07258113984652705


In [ ]:
inputs = [
    "як умру то пох",
    "Зоре моя вечірняя, зійди над горою,",
    "Чого мені тяжко, чого мені нудно, чого серце плаче, ридає, кричить,",
    "Якось-то йдучи уночі понад Невою… та, йдучи, міркую сам-таки з собою: 'якби-то,— думаю,",
    "Мені здається, я не знаю",
    "У тієї Катерини"
]

model.eval()  # Перемикаємо модель на режим оцінки
with torch.no_grad():  # Вимикаємо обчислення градієнтів
    for input_text in inputs:
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
        attention_mask = torch.ones(input_ids.shape, device=device)

        # Генерація тексту
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=200, temperature=0.7, top_p=0.9, top_k=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"Input: {input_text}")
        print(f"Generated: {generated_text}")
        print("="*50)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: як умру то пох
Generated: як умру то поховать, то поховайте, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі не вмру, Бо я в неволі


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: Зоре моя вечірняя, зійди над горою,
Generated: Зоре моя вечірняя, зійди над горою, І я в темнім садочку, і в темнім садочку, І в темнім садочку, і в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім садочку, І в темнім


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: Чого мені тяжко, чого мені нудно, чого серце плаче, ридає, кричить,
Generated: Чого мені тяжко, чого мені нудно, чого серце плаче, ридає, кричить, плаче, плаче, плаче, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, плачу, пла


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: Якось-то йдучи уночі понад Невою… та, йдучи, міркую сам-таки з собою: 'якби-то,— думаю,
Generated: Якось-то йдучи уночі понад Невою… та, йдучи, міркую сам-таки з собою: 'якби-то,— думаю,— я,— не знаю, де подіти,— а де подіти. А то, може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де подіти. А може, й не знаю, де подіти,— а де


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: Мені здається, я не знаю
Generated: Мені здається, я не знаю, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи то я, чи
Input: У тієї Катерини
Generated: У тієї Катерини, що вийшла заміж за Петра І, було двоє синів: Петро І, син Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катерина, що вийшла заміж за Петра І, та Катери